# Value iteration

![](imgs/value_iteration_1.png)

![](imgs/value_iteration_2.png)

![](imgs/value_iteration_3.png)

![](imgs/value_iteration_4.png)

![](imgs/value_iteration_5.png)

![](imgs/value_iteration_6.png)

In [94]:
import random

import numpy as np
import gym
import time
from gym import spaces
import os


# custom 2d grid world enviroment
class GridWorld(gym.Env):
    metadata = {'render.modes': ['console']}

    # actions available
    UP = 0
    LEFT = 1
    DOWN = 2
    RIGHT = 3


    def __init__(self, width, height):
        super(GridWorld, self).__init__()
        self.ACTION_NAMES = ["UP", "LEFT", "DOWN", "RIGHT"]
        self.num_actions = 4

        self.size = width * height  # size of the grid world
        self.num_states = self.size
        self.width = width
        self.height = height
        self.num_obstacles = int((width+height)/2)
        self.end_state = np.array([height - 1, width - 1], dtype=np.uint8) # goal state = bottom right cell

        # actions of agents : up, down, left and right
        self.action_space = spaces.Discrete(4)
        # observation : cell indices in the grid
        self.observation_space = spaces.MultiDiscrete([self.height, self.width])

        self.obstacles = np.zeros((height, width))

        for i in range(self.num_obstacles):
            self.obstacles[ random.randrange(height) , random.randrange(width)] = 1

        self.num_steps = 0
        self.max_steps = height*width

        self.current_state = np.zeros((2), np.uint8)#init state = [0,0]

        self.directions = np.array([
            [-1,0], #UP
            [0,-1], #LEFT
            [1,0], #DOWN
            [0,1] #RIGHT
        ])

    def transition_function(self, s, a):
        s_prime =  np.zeros((2), np.uint8)
        #s_prime = ?????
        s_prime = s + self.directions[a,:]

        if s_prime[0] < self.height and s_prime[1] < self.width and (s_prime >= 0).all():
            if self.obstacles[s_prime[0], s_prime[1]] == 0 :
                return s_prime

        return s

    def transition_probabilities(self, s, a):
        prob_next_state = np.zeros((self.heigth, self.width))
        s_prime = self.transition_function(s, a)

        prob_next_state[s_prime[0], s_prime[1]] = 1.0

        return prob_next_state#.flatten()

    def reward_function(self,s):
        r = 0
        #r= ????
        if (s == self.end_state).all():
            r = 1

        return r

    def termination_condition(self, s):
        done = False
        #done= ???

        done = (s == self.end_state).all() or self.num_steps > self.max_steps

        return done

    def step(self, action):
        s_prime = self.transition_function(self.current_state, action)
        reward = self.reward_function(s_prime)
        done = self.termination_condition(s_prime)

        self.current_state = s_prime
        self.num_steps += 1

        return self.current_state, reward, done, None

    def render(self):
        '''
            render the state
        '''

        row = self.current_state[0]
        col = self.current_state[1]

        for r in range(self.height):
            for c in range(self.width):
                if r == row and c == col:
                    print("| A ", end='')
                elif r == self.end_state[0] and c == self.end_state[1]:
                    print("| G ", end='')
                else:
                    if self.obstacles[r,c] == 1:
                        print('|///', end='')
                    else:
                        print('|___', end='')
            print('|')
        print('\n')



    def reset(self):
        self.current_state = np.zeros((2), np.uint8)
        self.num_steps = 0
        return self.current_state

    def reward_probabilities(self):
        rewards = np.zeros((self.num_states))
        i = 0
        for r in range(self.height):
            for c in range(self.width):
                state = np.array([r,c], dtype=np.uint8)
                rewards[i] = self.reward_function(state)
                i+=1

        return rewards

    def close(self):
        pass
    
    
class NonDeterministicGridWorld(GridWorld):
    def __init__(self, width, height, p=0.8):
        super(NonDeterministicGridWorld, self).__init__(width, height)
        self.probability_right_action = p

    def transition_function(self, s, a):
        s_prime = s + self.directions[a, :]

        #with probability 1 - p diagonal movement
        if random.random() <= 1 - self.probability_right_action:
            if random.random() < 0.5:
                s_prime = s_prime + self.directions[(a+1)%self.num_actions, :]
            else:
                s_prime = s_prime + self.directions[(a-1)%self.num_actions, :]


        if s_prime[0] < self.height and s_prime[1] < self.width and (s_prime >= 0).all():
            if self.obstacles[s_prime[0], s_prime[1]] == 0 :
                return s_prime

        return s

    def transition_probabilities(self, s, a):
        cells = []
        probs = []
        prob_next_state = np.zeros((self.height, self.width))
        s_prime_right =  s + self.directions[a, :]
        if s_prime_right[0] < self.height and s_prime_right[1] < self.width and (s_prime_right >= 0).all():
            if self.obstacles[s_prime_right[0], s_prime_right[1]] == 0 :
                prob_next_state[s_prime_right[0], s_prime_right[1]] = self.probability_right_action
                cells.append(s_prime_right)
                probs.append(self.probability_right_action)

        s_prime = s_prime_right + self.directions[(a + 1) % self.num_actions, :]
        if s_prime[0] < self.height and s_prime[1] < self.width and (s_prime >= 0).all():
            if self.obstacles[s_prime[0], s_prime[1]] == 0 :
                prob_next_state[s_prime[0], s_prime[1]] = (1 - self.probability_right_action) / 2
                cells.append(s_prime.copy())
                probs.append((1 - self.probability_right_action) / 2)

        s_prime = s_prime_right + self.directions[(a - 1) % self.num_actions, :]
        if s_prime[0] < self.height and s_prime[1] < self.width and (s_prime >= 0).all():
            if self.obstacles[s_prime[0], s_prime[1]] == 0 :
                prob_next_state[s_prime[0], s_prime[1]] = (1 - self.probability_right_action) / 2
                cells.append(s_prime.copy())
                probs.append((1 - self.probability_right_action) / 2)

        #normalization
        sump = sum(probs)
        #for cell in cells:
        #    prob_next_state[cell[0], cell[1]] /= sump
        prob_next_state[s[0], s[1]] = 1 - sump
        return prob_next_state


To apply value iteration we need the **transition probabilities** and the **reward function**

Print the probability over the next state

In [95]:
env = NonDeterministicGridWorld(3,5)
state = env.reset()
env.render()
#next state if we start from state 0,0 and we do action down
next_state_prob = env.transition_probabilities(state, 2)
print(next_state_prob)

| A |___|___|
|///|///|___|
|___|///|___|
|___|///|___|
|___|___| G |


[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


reward values

In [96]:
print(env.reward_probabilities())


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


# Value iteration algorithm

![](imgs/value_iteration.png)

In [97]:
def value_iteration(env, gamma=0.99):

    #initialize values
    values = np.zeros((env.num_states))
    STATES = np.zeros((env.num_states, 2), dtype=np.uint8)
    REWARDS = env.reward_probabilities()
    i = 0
    for r in range(env.height):
        for c in range(env.width):
            state = np.array([r, c], dtype=np.uint8)
            STATES[i] = state
            i += 1
    delta = 1
    i = 0
    while delta > 0.1:
        v_old = values.copy()
        for s in range(env.num_states):
            if s == env.num_states-1 or i >= env.max_steps:
                continue # if we reach the termination condition, we cannot perform any action
            state = STATES[s]
            max_va = -np.inf
            for a in range(env.num_actions):
                next_state_prob = env.transition_probabilities(state, a).flatten()
                va = (next_state_prob*(REWARDS + gamma*v_old))
                if s == env.num_states-1:
                    print(va, va.sum())
                
                va = va.sum()

                if va > max_va:
                    max_va = va
            values[s] = max_va
        print(values.reshape((env.height, env.width)))
        print()
        delta = abs(sum(v_old - values))
        i+=1
        #print(delta)
    return values

estimate values

In [98]:
values = value_iteration(env)
values[-1] = 1
values_r = values.reshape((env.height, env.width))
print(values_r)

[[0.  0.  0. ]
 [0.  0.  0. ]
 [0.  0.  0. ]
 [0.  0.1 0.8]
 [0.  0.8 0. ]]

[[0.     0.     0.    ]
 [0.     0.     0.    ]
 [0.     0.0792 0.6336]
 [0.0792 0.7336 0.9584]
 [0.6336 0.9584 0.    ]]

[[0.        0.        0.       ]
 [0.        0.0627264 0.5018112]
 [0.0627264 0.5966928 0.8845056]
 [0.6045336 0.9217792 0.9897632]
 [0.8845056 0.9897632 0.       ]]

[[0.         0.04967931 0.39743447]
 [0.04967931 0.48500052 0.79988705]
 [0.49121044 0.8481943  0.95902456]
 [0.85836382 0.97145851 0.99597311]
 [0.95902456 0.99597311 0.        ]]

[[0.03934601 0.39887517 0.71220257]
 [0.39887517 0.76779999 0.91792509]
 [0.77708381 0.93733761 0.97869757]
 [0.94312681 0.98375414 0.99720268]
 [0.97869757 0.99720268 0.        ]]

[[0.32369965 0.69442766 0.86801278]
 [0.69442766 0.89439579 0.95687764]
 [0.90081903 0.96472612 0.98356664]
 [0.96722109 0.98667558 0.99744613]
 [0.98356664 0.99744613 0.        ]]

[[0.61407924 0.85094535 0.92971362]
 [0.85094535 0.94115346 0.96844655]
 [0.94440127 0.9

Estimate best action

In [99]:
def best_action(env, values, s):
    max_va = 0
    best_a = -1
    for a in range(env.num_actions):
        expected_value_a = (env.transition_probabilities(s,a).flatten() * values).sum()
        
        ###### Expected value a
        
        #expected_value_a = ???
        
        ########
        print(env.ACTION_NAMES[a], expected_value_a)
        if expected_value_a > max_va:
            max_va = expected_value_a
            best_a = a
    
    return best_a

simulate optimal policy


In [100]:
done = False
state = env.reset()
while not done:
    action = best_action(env, values,state)
    
    state, reward, done, _ = env.step(action)
    env.render()


UP 0.9195317289207765
LEFT 0.9195317289207765
DOWN 0.9195317289207765
RIGHT 0.9422951150062722
|___| A |___|
|///|///|___|
|___|///|___|
|___|///|___|
|___|___| G |


UP 0.9479859615276461
LEFT 0.9252225754421505
DOWN 0.9504589253802889
RIGHT 0.9599721683672656
|___|___| A |
|///|///|___|
|___|///|___|
|___|///|___|
|___|___| G |


UP 0.9598775152613671
LEFT 0.9503642722743904
DOWN 0.970147983095533
RIGHT 0.9598775152613671
|___|___|___|
|///|///| A |
|___|///|___|
|___|///|___|
|___|___| G |


UP 0.9599721683672656
LEFT 0.9702426362014316
DOWN 0.9825948915953288
RIGHT 0.9727156000540744
|___|___|___|
|///|///|___|
|___|///| A |
|___|///|___|
|___|___| G |


UP 0.9751854229393879
LEFT 0.9850647144806424
DOWN 0.9950178565250765
RIGHT 0.9850647144806424
|___|___|___|
|///|///|___|
|___|///|___|
|___|///| A |
|___|___| G |


UP 0.9875529999917508
LEFT 0.997506142036185
DOWN 0.999501228407237
RIGHT 0.997506142036185
|___|___|___|
|///|///|___|
|___|///|___|
|___|///|___|
|___|___| A |


